In [1]:
import json
import os
import pandas as pd
import sys

In [2]:
os.getcwd()

'c:\\Users\\SMARTI25\\Dropbox\\TFG\\tfg-sb-meal-delivery-prediction-api\\clickhouse\\notebooks'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'c:\\Users\\SMARTI25\\Dropbox\\TFG\\tfg-sb-meal-delivery-prediction-api\\clickhouse'

In [5]:
from src.data.load_data import connect_clickhouse

In [6]:
# Get Clickhouse credentials and connect
filename_credentials = 'clickhouse_credentials.json'
with open(filename_credentials, 'r') as f:
    credentials = json.load(f)

client = connect_clickhouse(credentials)

In [7]:
# Create raw schema if not exists
database = 'raw'
client.execute('DROP DATABASE IF EXISTS ' + database)
client.execute('CREATE DATABASE IF NOT EXISTS ' + database)

[]

In [8]:
# Read initialize config file
filename_init_config = 'initialize_config.json'
with open(filename_init_config, 'r') as f:
    config = json.load(f)

In [9]:
config

{'files': [{'table_name': 'meal',
   'path': '../api/data/raw/meal_info.csv',
   'schema': [{'name': 'meal_id', 'type': 'Int64'},
    {'name': 'category', 'type': 'String'},
    {'name': 'cuisine', 'type': 'String'}]},
  {'table_name': 'center',
   'path': '../api/data/raw/fulfilment_center_info.csv',
   'schema': [{'name': 'center_id', 'type': 'Int64'},
    {'name': 'city_code', 'type': 'Int32'},
    {'name': 'region_code', 'type': 'Int32 '},
    {'name': 'center_type', 'type': 'String'},
    {'name': 'op_area', 'type': 'Float32'}]},
  {'table_name': 'train',
   'path': '../api/data/raw/train.csv',
   'schema': [{'name': 'id', 'type': 'Int64'},
    {'name': 'week', 'type': 'Int16'},
    {'name': 'center_id', 'type': 'Int64'},
    {'name': 'meal_id', 'type': 'Int64'},
    {'name': 'checkout_price', 'type': 'Float32'},
    {'name': 'base_price', 'type': 'Float32'},
    {'name': 'emailer_for_promotion', 'type': 'Int8'},
    {'name': 'homepage_featured', 'type': 'Int8'},
    {'name': 'num

In [12]:
# Read and create the following tables in raw
for t in config['files']:

    # Create the table
    ## First, get the schema and create a table using it in Clickhouse database
    schema = list()

    for s in t['schema']:
        schema.append(s['name'] + ' ' + s['type'])

    sentence = """
    CREATE TABLE IF NOT EXISTS """ + database + """.""" + t['table_name'] + """
    (""" + ', '.join(schema) + """)
    ENGINE = Memory
    """

    client.execute(sentence)

    # Read csv and insert data into its table
    df = pd.read_csv(t['path'])

    sentence = 'INSERT INTO ' + database + '.' + t['table_name'] + ' VALUES'
    client.insert_dataframe(sentence, df)